# Base de données: interroger une base de données MySQL

## Imports

In [1]:
from pathlib import Path
import pandas as pd
import PyPDF2

Préparation du dossier et vérification des fichiers

In [3]:
data_dir = Path("../data/camille_pdf")

# On vérifie que le dossier existe
if data_dir.exists():
    print(f"Le dossier existe deja:{data_dir}")
else:
     print(f"Le dossier n'existe pas. Creation de :{data_dir}")
     data_dir.mkdir(parents=True, exist_ok=True)
     print("Dossier cree avec succes")

Le dossier existe deja:..\data\camille_pdf


Exploration du dossier

In [4]:
pdf_files = sorted(data_dir.glob("*.pdf"))

if pdf_files:
    print(f"Nombre de fichiers PDF trouvés : {len(pdf_files)}")
    [f.name for f in pdf_files[:5]]
else:
    print("Aucun fichier PDF trouvé dans ce dossier. "
          "Fichers a telecharge a https://max.de.wilde.web.ulb.be/camille/")

Nombre de fichiers PDF trouvés : 51


Lecture d'un PDF et extraction du texte

Selction d'un PDF

In [5]:
assert len(pdf_files) > 0, "Aucun PDF trouvé dans ../data/camille_pdf"
sample_path = pdf_files[0]

reader = PyPDF2.PdfReader(sample_path)
num_pages = len(reader.pages)

print(f"Fichier : {sample_path.name}")
print(f"Pages   : {num_pages}")

Fichier : KB_JB230_1892-08-07_01-0003.pdf
Pages   : 1


Extraction et appercu du texte

In [8]:
texts = []
for p in range(num_pages):
    try:
        page = reader.pages[p]
        txt = page.extract_text() or ""
        texts.append(txt)
    except Exception as e:
        texts.append("")

full_text = "\n".join(texts)

print("Aperçu du texte extrait:\n")
print(full_text)

Aperçu du texte extrait:

llUr fili ont tiibi «veeS£
U section  nor- 
iceés l’examen  de sortie oour  
foWentiet  dn diplCmo  d’institairke.
Ce qui Km (frappé  quand  H noua a été donné  de 
ràier [ Institut,  c m le caractère  de distinction  qal 
ai rnpraac  aux élèves  par naedireettoart  ne g rores  
fe «alureaes  de premier  ordre.  la loa. ta tenue  août  
urftita. « ce a'eat  pas une choea banale  que de troo-  
ïer one Kaison  où l’on s'occupe,  en Même  tempe  qoe 
rtruer  Mer esprit  et de développer  leur intoffigaoaa,  
afcrncr  le caractère,  le Jugement  al les manière*  de*
Palais  de Justice
Don  M*
nard' 1asatlon rt. -ü„____ de Riom (France)  a prononcé  vendredi
m irrdt dan* l'affaire  d'assassinat  commis  i Ponl-  
f*ot. sur la personne  de IL Durif, par 1a femme  de „ dernier  et par l’ami de celle-ci.
artisanat,  autour  du crime, et aa compagne,  U 
veuve Durif,  sont  condamnés  4 mort.
[n entaadant  la condamnation  prononcée  contre  ara 
g, ii anime Carthonne

Analyse repide du texte

In [9]:
# on vérifie que 'full_text' existe (créé à l'étape 2)
assert 'full_text' in globals(), "Le texte n'a pas encore été extrait. Exécute d'abord l'etape precedenre."

nb_caracteres = len(full_text)
nb_mots = len(full_text.split())
moyenne_caracteres_par_page = nb_caracteres / num_pages if num_pages else 0
moyenne_mots_par_page = nb_mots / num_pages if num_pages else 0

print(f"Statistiques du document : {sample_path.name}\n")
print(f"Nombre de pages             : {num_pages}")
print(f"Nombre total de caractères  : {nb_caracteres:,}")
print(f"Nombre total de mots        : {nb_mots:,}")
print(f"Moyenne caractères / page   : {moyenne_caracteres_par_page:,.0f}")
print(f"Moyenne mots / page         : {moyenne_mots_par_page:,.0f}")

Statistiques du document : KB_JB230_1892-08-07_01-0003.pdf

Nombre de pages             : 1
Nombre total de caractères  : 42,150
Nombre total de mots        : 7,025
Moyenne caractères / page   : 42,150
Moyenne mots / page         : 7,025


Analyse comparative de plusieurs fichiers PDF

In [10]:
# On choisit 05 PDF pour éviter de tout lire
echantillon = pdf_files[:5]
resultats = []

for fichier in echantillon:
    reader = PyPDF2.PdfReader(fichier)
    num_pages = len(reader.pages)
    texte = ""
    for p in range(num_pages):
        try:
            page = reader.pages[p]
            texte += page.extract_text() or ""
        except:
            pass  # ici on ignore les pages mal reconnues

    nb_caracteres = len(texte)
    nb_mots = len(texte.split())
    moyenne_mots_par_page = nb_mots / num_pages if num_pages else 0

    resultats.append({
        "Fichier": fichier.name,
        "Pages": num_pages,
        "Caractères": nb_caracteres,
        "Mots": nb_mots,
        "Moyenne_mots/page": round(moyenne_mots_par_page, 1)
    })

In [11]:

# On cree un DataFrame pour visualiser les résultats
df_resultats = pd.DataFrame(resultats)
df_resultats

,Fichier,Pages,Caractères,Mots,Moyenne_mots/page
0,KB_JB230_1892-08-07_01-0003.pdf,1,42150,7025,7025.0
1,KB_JB230_1903-10-16_01-0002.pdf,1,50243,8083,8083.0
2,KB_JB230_1913-07-05_01-0001.pdf,1,39049,6349,6349.0
3,KB_JB258_1884-09-03_01-0003.pdf,1,32471,5270,5270.0
4,KB_JB258_1894-12-09_01-0003.pdf,1,41573,6755,6755.0


## Créer une base de données SQLite

In [2]:
# Créer un répertoire DB s'il n'existe pas encore
db_path = '../data/db'
if not os.path.exists(db_path):
    os.mkdir(db_path)

In [3]:
# Récupérer le fichier ZIP qui contient la DB de test
url = "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip"
filename = url.split("/")[-1]
# Récupérer le fichier zip dans la RAM
response = requests.get(url)

In [4]:
# Ecrire le fichier sur le disque
with open(os.path.join(db_path, filename), 'wb') as f:
    f.write(response.content)

In [5]:
# Extraire le zip pour obtenir la db
with zipfile.ZipFile(os.path.join(db_path, filename), 'r') as zip_ref:
    zip_ref.extractall(db_path)

## Schéma de la base de données

<img src="https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg" />

## Se connecter à la base de données

In [6]:
conn = sqlite3.connect(os.path.join(db_path, "chinook.db"))
c = conn.cursor()

## Trouver l'identifiant de Miles Davis

In [7]:
artist = 'Miles Davis'

query1 = f"""
    SELECT
        artistId
    FROM
        artists
    WHERE
        name = '{artist}';
"""
c.execute(query1)
artist_id = c.fetchone()[0]

print(artist_id)

68


## Trouver les identifiants d'albums de Miles Davis

In [8]:
query2 = f"""
    SELECT
        albumId
    FROM 
        albums 
    WHERE 
        artistId = '{artist_id}';
"""
albums_ids = []
for row in c.execute(query2):
    albums_ids.append(str(row[0]))

print(albums_ids)

['48', '49', '157']


## Trouver les morceaux des albums de Miles Davis

In [9]:
query3 = f"""
    SELECT
        name,
        albumId
    FROM
        tracks
    WHERE 
        albumId IN ({",".join(albums_ids)});
"""

songs = set()
for row in c.execute(query3):
    songs.add(row[0])

print(f"\n{len(songs)} distinct songs found:\n")
for song in sorted(songs):
    print(song)


36 distinct songs found:

'Round Midnight
Black Satin
Blues For Pablo
Blues For Pablo (Alternate Take)
Bye Bye Blackbird
Compulsion
E.S.P.
Generique
I Don't Wanna Be Kissed (By Anyone But You)
I Don't Wanna Be Kissed (By Anyone But You) (Alternate Take)
Jean Pierre (Live)
Jeru
Lament
Little Church (Live)
Miles Ahead
Miles Runs The Voodoo Down
My Funny Valentine (Live)
My Ship
Nefertiti
New Rhumba
Now's The Time
Petits Machins (Little Stuff)
Portia
So What
Someday My Prince Will Come
Springsville
Springsville (Alternate Take)
Summertime
Tempus Fugit
The Duke
The Maids Of Cadiz
The Meaning Of The Blues
The Meaning Of The Blues/Lament (Alternate Take)
The Pan Piper
Time After Time
Walkin'


## En une seule requête...

In [10]:
query4 = f"""
    SELECT
        tracks.name,
        tracks.albumId
    FROM
        artists
        JOIN albums ON artists.artistId = albums.artistId
        JOIN tracks ON albums.albumId = tracks.albumId
    WHERE 
        artists.name = '{artist}';
"""

songs = set()
for row in c.execute(query4):
    songs.add(row[0])

print(f"\n{len(songs)} distinct songs found:\n")
for song in sorted(songs):
    print(song)


36 distinct songs found:

'Round Midnight
Black Satin
Blues For Pablo
Blues For Pablo (Alternate Take)
Bye Bye Blackbird
Compulsion
E.S.P.
Generique
I Don't Wanna Be Kissed (By Anyone But You)
I Don't Wanna Be Kissed (By Anyone But You) (Alternate Take)
Jean Pierre (Live)
Jeru
Lament
Little Church (Live)
Miles Ahead
Miles Runs The Voodoo Down
My Funny Valentine (Live)
My Ship
Nefertiti
New Rhumba
Now's The Time
Petits Machins (Little Stuff)
Portia
So What
Someday My Prince Will Come
Springsville
Springsville (Alternate Take)
Summertime
Tempus Fugit
The Duke
The Maids Of Cadiz
The Meaning Of The Blues
The Meaning Of The Blues/Lament (Alternate Take)
The Pan Piper
Time After Time
Walkin'


## Pour en savoir plus

- [Tutoriel SQLite](https://www.sqlitetutorial.net/)
- [The SQL cheat sheet](https://www.sqltutorial.org/wp-content/uploads/2016/04/SQL-cheat-sheet.pdf)